# Timer

In [5]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [8]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
# pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '5m'
higher_timeframe = '1h'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [30]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [15]},
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 5m/1h, trade type is sell
Number of combinations is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.28s/it]


# Check local statistics

In [7]:
# 5m/1h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
74,STOCH_RSI,16,35,9,7,3,30,76.67,68.89,65.56,64.44,67.22,66.67,67.22,66.67,70.56,70.56,75.00,76.67,73.89,77.22,72.78,76.11,76.11,76.67,77.78,79.44,78.33,76.67,78.33,77.22,-0.21,-0.25,-0.28,-0.29,-0.27,-0.29,-0.38,-0.39,-0.41,-0.51,-0.64,-0.65,-0.68,-0.71,-0.68,-0.74,-0.66,-0.60,-0.66,-0.71,-0.80,-0.71,-0.70,-0.78,180,73.195000,-0.541667,575.100000,-97.500000
49,STOCH_RSI,14,35,9,7,3,30,77.53,70.48,66.52,64.32,66.52,65.20,64.32,64.32,66.96,65.64,69.60,71.37,70.48,74.89,72.25,76.21,76.21,75.33,77.53,78.85,77.97,77.53,78.41,77.53,-0.20,-0.23,-0.25,-0.26,-0.26,-0.25,-0.30,-0.30,-0.29,-0.34,-0.44,-0.50,-0.61,-0.65,-0.68,-0.75,-0.68,-0.66,-0.71,-0.73,-0.79,-0.73,-0.73,-0.82,227,71.915417,-0.506667,434.799583,-115.013333
19,STOCH_RSI,12,30,9,7,3,30,77.98,71.56,66.97,64.22,65.14,65.14,64.22,66.06,69.72,69.72,75.23,76.15,73.39,77.98,72.48,74.31,77.06,77.98,79.82,82.57,81.65,79.82,81.65,79.82,-0.24,-0.29,-0.26,-0.34,-0.34,-0.36,-0.42,-0.53,-0.50,-0.51,-0.70,-0.68,-0.70,-0.86,-0.83,-0.82,-0.79,-0.79,-0.79,-0.84,-0.86,-0.84,-0.77,-0.85,109,73.776667,-0.621250,411.656667,-67.716250
73,STOCH_RSI,16,35,9,7,3,25,78.26,70.81,65.84,62.73,66.46,63.98,64.60,64.60,68.94,69.57,74.53,76.40,73.29,77.02,71.43,75.16,75.16,75.78,76.40,78.26,77.64,76.40,78.26,77.02,-0.22,-0.29,-0.29,-0.26,-0.27,-0.27,-0.33,-0.32,-0.33,-0.44,-0.57,-0.63,-0.65,-0.69,-0.68,-0.73,-0.68,-0.61,-0.68,-0.73,-0.82,-0.73,-0.71,-0.81,161,72.439167,-0.530833,392.705833,-85.464167
24,STOCH_RSI,12,35,9,7,3,30,77.57,71.32,67.28,65.07,66.18,63.97,62.87,62.50,65.07,63.97,68.01,69.49,69.85,75.00,73.16,76.47,76.10,75.37,76.47,77.21,76.84,76.10,77.21,75.37,-0.19,-0.23,-0.23,-0.25,-0.23,-0.22,-0.24,-0.27,-0.29,-0.32,-0.33,-0.44,-0.49,-0.58,-0.60,-0.62,-0.60,-0.58,-0.62,-0.65,-0.68,-0.69,-0.68,-0.68,272,71.185417,-0.446250,322.433333,-121.380000
18,STOCH_RSI,12,30,9,7,3,25,78.00,73.00,68.00,63.00,64.00,63.00,62.00,64.00,68.00,68.00,74.00,75.00,72.00,77.00,71.00,74.00,77.00,78.00,79.00,82.00,81.00,79.00,81.00,79.00,-0.23,-0.31,-0.29,-0.27,-0.28,-0.27,-0.29,-0.38,-0.38,-0.43,-0.53,-0.61,-0.69,-0.83,-0.81,-0.81,-0.83,-0.79,-0.81,-0.84,-0.90,-0.84,-0.80,-0.83,100,72.916667,-0.585417,291.666667,-58.541667
48,STOCH_RSI,14,35,9,7,3,25,78.43,71.57,66.18,62.25,65.69,62.75,62.25,62.75,65.69,64.71,68.63,70.59,69.61,74.51,71.57,75.98,75.98,75.00,76.96,78.43,77.94,77.45,78.92,77.94,-0.20,-0.24,-0.25,-0.19,-0.21,-0.21,-0.26,-0.28,-0.29,-0.32,-0.41,-0.49,-0.59,-0.65,-0.71,-0.75,-0.69,-0.69,-0.75,-0.77,-0.82,-0.77,-0.74,-0.87,204,71.324167,-0.506250,270.130000,-103.275000
44,STOCH_RSI,14,30,9,7,3,30,77.50,67.50,62.50,62.50,63.75,65.00,62.50,65.00,67.50,68.75,75.00,73.75,70.00,73.75,73.75,75.00,75.00,77.50,81.25,86.25,85.00,82.50,83.75,81.25,-0.22,-0.23,-0.20,-0.30,-0.39,-0.60,-0.66,-0.71,-0.66,-0.70,-0.90,-0.78,-0.77,-0.95,-1.04,-0.93,-0.87,-0.82,-0.86,-0.88,-0.90,-0.79,-0.71,-0.86,80,73.177083,

In [3]:
# 15m/1h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
51,STOCH_RSI,16,15,9,7,3,15,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,75.00,100.00,75.00,75.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,-2.15,-2.57,-2.03,-2.61,-3.35,-3.19,-2.86,-3.30,-4.08,-3.91,-3.53,-3.92,-3.16,-2.71,-2.40,-2.45,-2.68,-2.19,-1.81,-2.44,-3.40,-3.72,-3.67,-4.26,4,96.875000,-3.016250,107.500000,-12.065000
52,STOCH_RSI,16,15,9,7,3,20,100.00,100.00,100.00,100.00,100.00,80.00,100.00,80.00,80.00,60.00,80.00,60.00,80.00,80.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,80.00,100.00,100.00,-2.15,-2.57,-1.99,-2.16,-2.80,-3.17,-2.80,-3.12,-3.65,-3.65,-3.44,-3.88,-3.02,-2.43,-2.12,-1.80,-2.22,-2.10,-1.22,-1.53,-3.17,-3.62,-3.06,-3.06,5,90.833333,-2.697083,104.166667,-13.485417
53,STOCH_RSI,16,15,9,7,3,25,100.00,100.00,100.00,100.00,100.00,80.00,100.00,80.00,80.00,60.00,80.00,60.00,80.00,80.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,80.00,100.00,100.00,-2.15,-2.57,-1.99,-2.16,-2.80,-3.17,-2.80,-3.12,-3.65,-3.65,-3.44,-3.88,-3.02,-2.43,-2.12,-1.80,-2.22,-2.10,-1.22,-1.53,-3.17,-3.62,-3.06,-3.06,5,90.833333,-2.697083,104.166667,-13.485417
54,STOCH_RSI,16,15,9,7,3,30,100.00,100.00,100.00,100.00,100.00,80.00,100.00,80.00,80.00,60.00,80.00,60.00,80.00,80.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,80.00,100.00,100.00,-2.15,-2.57,-1.99,-2.16,-2.80,-3.17,-2.80,-3.12,-3.65,-3.65,-3.44,-3.88,-3.02,-2.43,-2.12,-1.80,-2.22,-2.10,-1.22,-1.53,-3.17,-3.62,-3.06,-3.06,5,90.833333,-2.697083,104.166667,-13.485417
26,STOCH_RSI,14,15,9,7,3,15,100.00,83.33,83.33,100.00,100.00,83.33,83.33,100.00,83.33,66.67,83.33,66.67,66.67,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,-1.92,-1.92,-1.84,-1.69,-2.29,-2.30,-2.24,-2.39,-2.54,-2.57,-2.99,-3.21,-2.74,-2.48,-2.40,-2.45,-2.68,-2.19,-1.81,-2.43,-3.25,-3.51,-3.11,-3.23,6,84.025833,-2.507500,84.155000,-15.045000
27,STOCH_RSI,14,15,9,7,3,20,100.00,85.71,85.71,100.00,100.00,71.43,85.71,85.71,71.43,57.14,71.43,57.14,71.43,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,71.43,85.71,85.71,-1.76,-1.27,-1.69,-1.23,-1.78,-1.42,-1.67,-1.67,-1.42,-1.48,-2.53,-2.53,-2.47,-2.43,-2.12,-1.80,-2.22,-2.10,-1.22,-1.53,-3.17,-3.40,-3.06,-3.06,7,81.545417,-2.042917,80.817917,-14.300417
29,STOCH_RSI,14,15,9,7,3,30,100.00,85.71,85.71,100.00,100.00,71.43,85.71,85.71,71.43,57.14,71.43,57.14,71.43,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,71.43,85.71,85.71,-1.76,-1.27,-1.69,-1.23,-1.78,-1.42,-1.67,-1.67,-1.42,-1.48,-2.53,-2.53,-2.47,-2.43,-2.12,-1.80,-2.22,-2.10,-1.22,-1.53,-3.17,-3.40,-3.06,-3.06,7,81.545417,-2.042917,80.817917,-14.300417
28,STOCH_RSI,14,15,9,7,3,25,100.00,85.71,85.71,100.00,100.00,71.43,85.71,85.71,71.43,57.14,71.43,57.14,71.43,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,71.43,85.71,85.71,-1.76,-1.27,-1.69,-1.23,-1.78,-1.42,-1.67,-1.67,-1.42,-1.48,-2.53,-2.53,-2.47,-2.43,-2.12,-1.80,-2

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [15]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [15]},
                'LinearReg': {'timeperiod': [6], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            